In [1]:
import tensorflow as tf
import numpy as np
import pickle
import os
import shutil

In [2]:
batch_size = 64
img_height = 120
img_width = 160

num_classes = 2

In [3]:
def preprocess_input(image):
    return (image/127.5) - 1

def get_model(lr=0.001, alpha=0.35):
    base_model = tf.keras.applications.MobileNetV2(
        input_shape=(img_height, img_width, 3),
        alpha=alpha,
        include_top=False, 
        weights='imagenet')
    base_model.trainable = False
    # Add a new classifier layer for transfer learning
    global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
    dropout = tf.keras.layers.Dropout(0.2)
    prediction_layer = tf.keras.layers.Dense(num_classes, activation='softmax')
  
    model = tf.keras.Sequential([
        base_model,
        global_average_layer,
        dropout,
        prediction_layer
    ])
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=lr, momentum=0.9),
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
    return model

def load_model(checkpoint_path):
    model = get_model()
    # load weights
    model.load_weights(checkpoint_path)
    return model

def predict(img_path, model):
    test_ds = tf.keras.preprocessing.image_dataset_from_directory(
          img_path,
          shuffle=False,
          interpolation="bilinear",
          image_size=(img_height, img_width),
          batch_size=batch_size,
    )

    test_ds_2 = test_ds.map(
      lambda x, y: (preprocess_input(x), y))

    with open(CLASS_PEOPLE + '.pkl','rb') as f:
        class_to_index = pickle.load(f)

    result = model.predict(test_ds_2, batch_size=batch_size)
    loss,acc = model.evaluate(test_ds_2)
#     print(acc)
#     for item in result:
#         print(class_to_index[np.argmax(item)])
        
    scores = []
    for i in range(len(result)):
        scores.append(result[i][1])
#     print(scores)
    return result, test_ds, scores

In [4]:
def find_fn_fp(y_r, y_p, score, img_path, dest_path):
    if y_r!=y_p and y_p==0: #0:negative, this belongs to FN
        img_name = os.path.basename(img_path)
        new_img_name = '.'.join(img_name.split('.')[:-1])+'__'+'%.3f'%score+'.jpg'
        shutil.copy(img_path, dest_path+'/FN/'+new_img_name)
    elif y_r!=y_p and y_p==1: #0:negative, this belongs to FN
        img_name = os.path.basename(img_path)
        new_img_name = '.'.join(img_name.split('.')[:-1])+'__'+'%.3f'%score+'.jpg'
        shutil.copy(img_path, dest_path+'/FP/'+new_img_name)
    if y_r==y_p and y_p==0: #0:negative, this belongs to TN
        img_name = os.path.basename(img_path)
        new_img_name = '.'.join(img_name.split('.')[:-1])+'__'+'%.3f'%score+'.jpg'
        shutil.copy(img_path, dest_path+'/TN/'+new_img_name)
    elif y_r==y_p and y_p==1: #0:negative, this belongs to TP
        img_name = os.path.basename(img_path)
        new_img_name = '.'.join(img_name.split('.')[:-1])+'__'+'%.3f'%score+'.jpg'
        shutil.copy(img_path, dest_path+'/TP/'+new_img_name)

In [5]:
def cal_precision_recall(result, test_ds, scores, images, dest):
    y_p = []
    for item in result:
        y_p.append(np.argmax(item))
        
    y_r = []
    for x, y in test_ds:
        y_n = y.numpy()
        y_r.extend(y_n)

    m_p = tf.keras.metrics.Precision()
    m_r = tf.keras.metrics.Recall()

    m_p.update_state(y_r, y_p)
    m_r.update_state(y_r, y_p)

    precision = m_p.result().numpy()
    recall = m_r.result().numpy()
    print("precision:%s\nrecall:%s"%(precision, recall))
    
    for i in range(len(y_p)):
        find_fn_fp(y_r[i], y_p[i], scores[i], images[i], dest)

In [4]:
# img_path = 'test'
img_path = '../data/12_08_xinhu/12_8-12_9_initial/1/labeled'
model_path = "training_12_17/cp-0070.ckpt"
CLASS_PEOPLE = 'checkpoints/people_class'

model = load_model(model_path)
result, test_ds, scores = predict(img_path, model)
y_p = []
for item in result:
    y_p.append(np.argmax(item))
# print(y_p)

y_r = []
for x, y in test_ds:
    y_n = y.numpy()
    y_r.extend(y_n)
    
m_p = tf.keras.metrics.Precision()
m_r = tf.keras.metrics.Recall()
# m_auc = tf.keras.metrics.AUC()

m_p.update_state(y_r, y_p)
m_r.update_state(y_r, y_p)
# m_auc.update_state(y_r, y_p)

precision = m_p.result().numpy()
recall = m_r.result().numpy()
# auc = m_auc().result.numpy()
print("precision:%s\nrecall:%s"%(precision, recall))

Found 13372 files belonging to 2 classes.
209/209 [==============================] - 54s 258ms/step - loss: 0.3346 - accuracy: 0.9950
precision:0.99500555
recall:0.98651993


In [6]:
import os
import shutil

DATA_PATH = '../../Desktop/labeled_data/05_23_wuxi/'
model_path = "../../Desktop/model/21_05_25/Best_Model.ckpt"
CLASS_PEOPLE = 'checkpoints/people_class'
dest_path = "../../Desktop/results/21_05_23_wuxi/"

# seed = 12
# TEST_PATH = '../../training/val_01_18_relabel_2'
# test_dest_path = '../../results/val_01_18_relabel_2/'


if not os.path.exists(dest_path+'FP'):
    os.makedirs(dest_path+'FP')
if not os.path.exists(dest_path+'FN'):
    os.makedirs(dest_path+'FN')
if not os.path.exists(dest_path+'TP'):
    os.makedirs(dest_path+'TP')
if not os.path.exists(dest_path+'TN'):
    os.makedirs(dest_path+'TN')


with open(CLASS_PEOPLE + '.pkl','rb') as f:
    class_to_index = pickle.load(f)
    print(class_to_index)

model = load_model(model_path)
for class_dir in os.listdir(DATA_PATH):
    for img in os.listdir(DATA_PATH+'/'+class_dir):
        print(img)
        img_path = DATA_PATH+'/'+class_dir+'/'+img

        img_pil = tf.keras.preprocessing.image.load_img(
            img_path,
            target_size=(img_height, img_width),
            interpolation='bilinear')
        # resize
#         img_pil = img_pil.resize((resized_width, resized_height))

        img_array = tf.keras.preprocessing.image.img_to_array(img_pil)
#         print(img_array.shape)
        img_array = tf.expand_dims(img_array, 0)
        img_array = preprocess_input(img_array)

        result = model.predict(img_array)
        print(result)
        c = class_to_index[np.argmax(result)]
#         print(class_to_index[np.argmax(result)])
        
        if c!=class_dir and c=='negative':
            new_img = img.split('.')[0]+'--'+'%.3f'%result[0][1]+'.jpg'
            print(img_path)
            print(result)
            shutil.copy(img_path, dest_path+'FN/'+new_img)
        elif c!=class_dir and c=='positive':
            new_img = img.split('.')[0]+'--'+'%.3f'%result[0][1]+'.jpg'
            print(img_path)
            print(result)
            shutil.copy(img_path, dest_path+'FP/'+new_img)
        elif c==class_dir and c=='positive':
            new_img = img.split('.')[0]+'--'+'%.3f'%result[0][1]+'.jpg'
            print(img_path)
            print(result)
            shutil.copy(img_path, dest_path+'TP/'+new_img)
        elif c==class_dir and c=='negative':
            new_img = img.split('.')[0]+'--'+'%.3f'%result[0][1]+'.jpg'
            print(img_path)
            print(result)
            shutil.copy(img_path, dest_path+'TN/'+new_img)

['negative', 'positive']

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb126f44460> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb126f44bb0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb126f44bb0> and <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb126f44970>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.DepthwiseConv2D object at 0x7fb126fc3ca0> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1270a4d00>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1270a4d00> and <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb1270a4b50>).

Two che


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1254a5580> and <tensorflow.python.keras.layers.merge.Add object at 0x7fb125c6fca0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb1259c6e50> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1270a4310>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1270a4310> and <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb125497dc0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.DepthwiseConv2D object at 0x7fb13b09a550> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb13b09a340>).

Two checkpoint references resolved to different 


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb127274f10> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb127261550>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb127261550> and <tensorflow.python.keras.layers.merge.Add object at 0x7fb127249a00>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb12727b3d0> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb127282c10>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb127282c10> and <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb12726de20>).

Two checkpoint references resolved to different objects (


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1272b31f0> and <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb1272b8250>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb1272b88b0> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1272be760>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1272be760> and <tensorflow.python.keras.layers.merge.Add object at 0x7fb1272be310>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb1272c80d0> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1272ce2b0>).

Two checkpoint references resolved to different objects (


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.DepthwiseConv2D object at 0x7fb1272ce910> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1272e2a00>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb1272e2a00> and <tensorflow.python.keras.layers.advanced_activations.ReLU object at 0x7fb127502ca0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x7fb1272a4640> and <tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb12722c040>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.normalization_v2.BatchNormalization object at 0x7fb12722c040> and <tensorflow.python.keras.layers.merge.Add object at 0x7fb127271d00>).

Two checkpoint references resolved to different 

...

[[1.000000e+00 6.857933e-25]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--18_09_31--010__0.000.jpg
[[1.000000e+00 6.857933e-25]]
img_R2N09333-202105_24--05_11_26--011__0.000.jpg
[[1.0000000e+00 2.3855878e-23]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--05_11_26--011__0.000.jpg
[[1.0000000e+00 2.3855878e-23]]
img_R2N09333-202105_23--10_40_16--009__0.000.jpg
[[1.000000e+00 6.234562e-23]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--10_40_16--009__0.000.jpg
[[1.000000e+00 6.234562e-23]]
img_R2N09333-202105_23--08_01_48--008__0.000.jpg
[[1.000000e+00 6.779462e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--08_01_48--008__0.000.jpg
[[1.000000e+00 6.779462e-09]]
img_R2N09333-202105_23--23_15_38--009__0.000.jpg
[[1.000000e+00 4.487239e-08]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--23_15_38--009__0.000.jpg
[[1.000000e+00 4.487239e-08]]
img_R2N09333-20

[[1.0000000e+00 4.3136463e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--22_26_28--007__0.000.jpg
[[1.0000000e+00 4.3136463e-09]]
img_R2N09333-202105_24--15_33_47--016__0.000.jpg
[[1.000000e+00 6.621159e-22]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--15_33_47--016__0.000.jpg
[[1.000000e+00 6.621159e-22]]
img_R2N09333-202105_23--04_43_27--008__0.000.jpg
[[1.00000e+00 7.77361e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--04_43_27--008__0.000.jpg
[[1.00000e+00 7.77361e-09]]
img_R2N09333-202105_24--03_37_53--011__0.000.jpg
[[1.000000e+00 9.182046e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--03_37_53--011__0.000.jpg
[[1.000000e+00 9.182046e-09]]
img_R2N09333-202105_24--01_19_13--008__0.000.jpg
[[1.0000000e+00 4.4449435e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--01_19_13--008__0.000.jpg
[[1.0000000e+00 4.4449435e-09]]
img_R2N09333-20

[[1.0000000e+00 4.4813803e-23]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--12_30_33--012__0.000.jpg
[[1.0000000e+00 4.4813803e-23]]
img_R2N09333-202105_23--00_39_11--013__0.000.jpg
[[1.0000000e+00 1.6062142e-24]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--00_39_11--013__0.000.jpg
[[1.0000000e+00 1.6062142e-24]]
img_R2N09333-202105_23--07_20_12--010__0.000.jpg
[[1.0000000e+00 7.7045836e-25]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--07_20_12--010__0.000.jpg
[[1.0000000e+00 7.7045836e-25]]
img_R2N09333-202105_23--21_59_18--012__0.000.jpg
[[1.0000000e+00 5.9684018e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--21_59_18--012__0.000.jpg
[[1.0000000e+00 5.9684018e-09]]
img_R2N09333-202105_23--23_49_26--007__0.000.jpg
[[1.0000000e+00 1.1511693e-10]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--23_49_26--007__0.000.jpg
[[1.0000000e+00 1.1511693e-10]]

[[1.000000e+00 5.845376e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--22_26_48--006__0.000.jpg
[[1.000000e+00 5.845376e-09]]
img_R2N09333-202105_24--00_31_56--008__0.000.jpg
[[1.0000000e+00 1.3516719e-22]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--00_31_56--008__0.000.jpg
[[1.0000000e+00 1.3516719e-22]]
img_R2N09333-202105_23--01_05_07--008__0.000.jpg
[[1.0000000e+00 1.2574036e-24]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--01_05_07--008__0.000.jpg
[[1.0000000e+00 1.2574036e-24]]
img_R2N09333-202105_23--21_58_08--013__0.000.jpg
[[1.000000e+00 5.093604e-10]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--21_58_08--013__0.000.jpg
[[1.000000e+00 5.093604e-10]]
img_R2N09333-202105_24--04_34_43--009__0.000.jpg
[[1.0000000e+00 2.8875607e-10]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--04_34_43--009__0.000.jpg
[[1.0000000e+00 2.8875607e-10]]
img_R2N

[[1.0000000e+00 3.7031094e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--02_49_13--010__0.000.jpg
[[1.0000000e+00 3.7031094e-09]]
img_R2N09333-202105_23--19_10_01--013__0.000.jpg
[[1.0000000e+00 5.2862386e-24]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_23--19_10_01--013__0.000.jpg
[[1.0000000e+00 5.2862386e-24]]
img_R2N09333-202105_24--06_51_23--007__0.000.jpg
[[1.0000000e+00 5.9985966e-10]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--06_51_23--007__0.000.jpg
[[1.0000000e+00 5.9985966e-10]]
img_R2N09333-202105_24--04_57_43--009__0.000.jpg
[[1.000000e+00 9.124081e-10]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--04_57_43--009__0.000.jpg
[[1.000000e+00 9.124081e-10]]
img_R2N09333-202105_24--02_03_52--013__0.000.jpg
[[1.00000e+00 4.03405e-09]]
../../Desktop/labeled_data/05_23_wuxi//negative/img_R2N09333-202105_24--02_03_52--013__0.000.jpg
[[1.00000e+00 4.03405e-09]]
img_R2N0933